In [ ]:
import cirq
import numpy as np
import time
from qsimcirq import QSimSimulator

# original
def cirq_counts(circuit: cirq.Circuit, shots=1024):
    qubits = list(circuit.all_qubits())
    circuit = circuit.copy()
    circuit.append(cirq.measure(*qubits, key='m'))

    simulator = QSimSimulator()
    result = simulator.run(circuit, repetitions=int(shots)) 
    hist = result.histogram(key='m')

    counts = {format(k, f'0{len(qubits)}b'): v for k, v in hist.items()}
    return counts

# optimized
def cirq_counts_faster(circuit: cirq.Circuit, shots=1024):
    qubits = list(circuit.all_qubits())
    width = len(qubits)

    circuit = circuit.copy()
    circuit.append(cirq.measure(*qubits, key='m'))

    simulator = QSimSimulator()
    result = simulator.run(circuit, repetitions=int(shots))
    bitstrings = result.measurements['m']  # dimension of matrix is (shots, width)

    # fast histogram using vectorized bitstring to integer conversion
    powers = 1 << np.arange(width)[::-1]  # bit shift -> [2^(n-1), ..., 1], len(powers) equals width
    keys = np.dot(bitstrings, powers) # binary to integer
    counts_array = np.bincount(keys, minlength=2 ** width) # count up occurance of integer

    return {k: v for k, v in enumerate(counts_array) if v > 0}


def build_deep_circuit():
    # 12 qubit
    qubits = cirq.LineQubit.range(12)
    circuit = cirq.Circuit()
    depth = 0

    # 150 gates depth
    while depth < 150:
        for q in qubits:
            circuit.append(cirq.rx(np.pi / 4)(q))
            circuit.append(cirq.ry(np.pi / 8)(q))
            circuit.append(cirq.H(q))
            depth += 3
            if depth >= 150:
                break
        for i in range(0, 11, 2):
            circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))
            depth += 1
            if depth >= 150:
                break
    return circuit

def hellinger_fidelity(counts1, counts2):
# Verified and debugged. See code_verification\hellinger_fidelity.ipynb
    """Compute Hellinger fidelity between two count dictionaries."""
    if not counts1 or not counts2:
        return 0.0  # fallback if any input is empty

    all_keys = set(counts1.keys()).union(counts2.keys())
    shots1 = sum(counts1.values())
    shots2 = sum(counts2.values())

    if shots1 == 0 or shots2 == 0:
        return 0.0  # avoid divide by zero

    p = np.array([np.sqrt(counts1.get(k, 0) / shots1) for k in all_keys])
    q = np.array([np.sqrt(counts2.get(k, 0) / shots2) for k in all_keys])

    fidelity = np.sum(p * q) ** 2
    return fidelity

def benchmark():
    for _ in range(3):
        circuit = build_deep_circuit()
        shots = (2**12)*100

        start = time.time()
        normal = cirq_counts(circuit, shots)
        print("Fast count:", round(time.time() - start, 4), "s")

        start = time.time()
        faster = cirq_counts_faster(circuit, shots)
        print("Faster count:", round(time.time() - start, 4), "s")
        
    # print(normal)
    # print(faster)

    # faster counter uses no bit string formatting for efficiency
    # Must be compared to another outcome from the same function (not comparable to normal)
    faster1 = cirq_counts_faster(circuit, shots)
    print("Fidelity:")
    print(hellinger_fidelity(faster,faster1))

if __name__ == "__main__":
    benchmark()


Fast count: 3.1323 s
Faster count: 2.2984 s
Fast count: 3.2641 s
Faster count: 1.915 s
Fast count: 3.2418 s
Faster count: 1.9224 s
Fidelity:
0.9951320227306151
